In [1]:
import numpy as np
from six.moves import zip

from bokeh.plotting import figure, show, output_notebook, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [2]:
import os
import sys
module_path = os.path.abspath((os.path.join(os.getcwd(), '..')))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np

from src.data import util
from joblib import load, dump
import pickle

PATH = '/Users/lucasbaier/OneDrive - KIT/Lucas Baier/Code/TaxiData_Weather/'

Using TensorFlow backend.
/Users/lucasbaier/anaconda3/envs/taxiData/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lucasbaier/anaconda3/envs/taxiData/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lucasbaier/anaconda3/envs/taxiData/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [3]:
with open(PATH + 'results/all_predictions.pickle', 'rb') as handle:
    predictions = pickle.load(handle)

In [4]:
df = util.read_all(show=False)
regions = util.largest(20)

In [5]:
def compute_errors(df, name, pred):
    results = {}
    _pred = pred
    results['Rmse'] = util.rmse(df, _pred)
    results['Smape'] = util.smape(df, _pred)
    results['Name'] = name
    return results

In [6]:
results = [compute_errors(df, index, item) for index, item in predictions.items()]

In [7]:
results_df = pd.DataFrame.from_dict(results).set_index('Name')

In [8]:
p = figure(x_range=results_df.index.values, plot_height=350, title="Results", y_range = (56,62),
           toolbar_location=None, tools="")

p.vbar(x=results_df.index.values, top=results_df.Rmse.values, width=0.9)
#p.vbar(x=results_df.index.values, top=results_df.Smape.values, width=0.9)

p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 7
#p.y_range.start = 56
#p.y_range.end = 62

show(p)

In [9]:
results_df.index.values

array(['static_no_weather_nh_2009-2011', 'static_weather_nh_2009-2011',
       'static_no_weather_h_2009-2011', 'static_weather_h_2009-2011',
       'yearly_training_no_weather_3y', 'yearly_training_weather_3y',
       'yearly_training_no_weather_h_3y', 'yearly_training_weather_h_3y'],
      dtype=object)

In [10]:
def compute_yearly_errors(df, name, pred):
    _pred = pred
    rmse = []
    smape = []
    year = []
    results = {}
    for i in range(2012, 2019):
        pred_year = _pred.loc[_pred.index.year == i]
        rmse.append(util.rmse(df, pred_year))
        smape.append(util.smape(df, pred_year))
        year.append(i)

    results['Name'] = name
    results['Rmse'] = rmse
    results['Smape'] = smape
    results['Year'] = year
    return(results)

In [11]:
results = [compute_yearly_errors(df, index, item) for index, item in predictions.items()]

In [12]:
def create_df_yearly(errors, metric):
    results_metric = [pd.Series(data = i[metric], index = i['Year'], name= i['Name']) for i in errors]
    df = pd.concat(results_metric, axis = 1)
    return df


In [13]:
results_df_yearly = create_df_yearly(results, 'Rmse')
results_df_yearly['Year'] = results_df_yearly.index
source = ColumnDataSource(results_df_yearly)

In [14]:
from bokeh.palettes import Spectral8

p = figure(plot_width=600, plot_height=400)

for i, name in enumerate(results_df_yearly.columns[:-1]):
    p.line(x = 'Year', y=name, line_width=2, source=source, legend_label=name, color = Spectral8[i])

p.yaxis.axis_label = 'RMSE'
p.title.text = 'Development of RMSE over time'

p.legend.location = 'bottom_left'
p.legend.click_policy="hide"

show(p)

In [15]:
results_df_yearly_smape = create_df_yearly(results, 'Smape')
results_df_yearly_smape['Year'] = results_df_yearly_smape.index
source_smape = ColumnDataSource(results_df_yearly_smape)

In [16]:
from bokeh.palettes import Spectral8

p = figure(plot_width=600, plot_height=400)

for i, name in enumerate(results_df_yearly.columns[:-1]):
    p.line(x = 'Year', y=name, line_width=2, source=source_smape, legend_label=name, color = Spectral8[i])

p.yaxis.axis_label = 'Smape'
p.title.text = 'Development of Smape over time'

p.legend.location = 'top_left'
p.legend.click_policy="hide"

show(p)